# CKP8366 - TÓPICOS AVANÇADOS - APRENDIZAGEM DE MÁQUINA PROBABILÍSTICA

<img  src="https://img.shields.io/badge/UFC_CKP8366-VAUX GOMES-000000?style=for-the-badge&logo=" /> <img src="https://img.shields.io/badge/Jupyter-000000?style=for-the-badge&logo=jupyter&logoColor=white" /> <img src="https://img.shields.io/badge/Python-000000?style=for-the-badge&logo=python&logoColor=white" />


In [1]:
%pip install --upgrade pip
%pip install torch # Pode demorar um pouco

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import torch
import matplotlib.pyplot as plt

## Radial Basis Function Kernel

- Versão mais comum:
$$k(x_1, x_2) = \exp\left(-\frac{\|x_1 - x_2\|^2}{2\sigma^2}\right)$$

- O slide mostra uma versão ponderada (um pouco diferente)

> $$\displaystyle k(x_i, x_j) = \sigma^2 \exp \left(-\frac{1}{2} \sum_{d=1}^D w_d^2(x_{id} - x_{jd})^2\right)$$

In [11]:
# RBF
def rbf(xi, xj, sigma, weights=None):
  ''' Funciona tanto para Matriz x Vetor quanto para Matriz x Matriz '''
  xi = torch.tensor(xi, dtype=torch.float32)
  xj = torch.tensor(xj, dtype=torch.float32)
 
  diff = torch.cdist(xi, xj, p=2)
  return sigma**2 * torch.exp(-0.5 * (diff ** 2 * weights**2))

## Gaussian Process

In [22]:
class GaussianProcessRBF:
  def __init__(self, sigma=1.0, noise=0.1, learning_rate=0.01, epochs=100):
    self.lr = learning_rate
    self.epochs = max(10, epochs) # Safety
    
    self.sigma = torch.tensor(sigma, dtype=torch.float32, requires_grad=True)
    self.noise = torch.tensor(noise, dtype=torch.float32, requires_grad=True)
  
  # RBF Kernel
  def kernel(self, xi, xj):
    # Safety
    xi = torch.tensor(xi, dtype=torch.float32) if not isinstance(xi, torch.Tensor) else xi
    xj = torch.tensor(xj, dtype=torch.float32) if not isinstance(xj, torch.Tensor) else xj
    
    #
    sqdist = torch.sum((xi.unsqueeze(1) - xj.unsqueeze(0))**2 * self.weights**2, dim=-1)
    return self.sigma**2 * torch.exp(-0.5 * sqdist)
  
  # 
  def fit(self, X, y):
    # Train data is required for prediction  :-(
    self.X = torch.tensor(X, dtype=torch.float32)
    self.y = torch.tensor(y, dtype=torch.float32)
    
    #
    self.weights = torch.randn(self.X.shape[1], dtype=torch.float32, requires_grad=True)
    
    #
    optim = torch.optim.Adam([self.sigma, self.noise, self.weights], lr=self.lr)
    
    # Epochs
    for e in range(self.epochs):
      # Gradient reset
      optim.zero_grad()
      
      #
      K = self.kernel(self.X, self.X) + self.noise**2 * torch.eye(len(self.X))
      loss = torch.mean(torch.logdet(K) + self.y.T @ torch.linalg.solve(K, self.y)) # solve == inv
      
      #
      loss.backward()
      optim.step()
    
    #
    self.K_inv = torch.linalg.solve(
      self.kernel(self.X, self.X) + self.noise**2 * torch.eye(len(X)), self.y) # solve == inv
  
  #
  def predict(self, X):
    X = torch.tensor(X, dtype=torch.float32)
    
    K_  = self.kernel(self.X, X)                                #
    K__ = self.kernel(X, X) + self.noise**2 * torch.eye(len(X)) #
    
    mu    = K_.T @ self.K_inv @ self.y
    cov   = K__ - K_.T @ self.K_inv @ K_
    sigma = torch.sqrt(torch.diag(cov))
    
    #
    return mu.detach().numpy(), sigma.detach().numpy()

  def __str__(self):
    return f'GaussianProcessRBF'

#### Carregamento dos dados

In [13]:
def prepare_data(path, delimiter=','):
  # Data
  data = np.genfromtxt(path, delimiter=',')

  # Separation
  X = data[:, :-1]
  y = data[:, -1:]

  # Normalization 
  X = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))           # MinMax  0 ~ 1
  y = 2  * (y - y.min(axis=0)) / (y.max(axis=0) - y.min(axis=0)) - 1  # MinMax -1 ~ 1            

  return X, y

#
X_train, y_train = prepare_data('./files/gp_data_train.csv')
X_test,  y_test  = prepare_data('./files/gp_data_test.csv')

#
X_train.shape, y_train.shape, X_test.shape,  y_test.shape

((100, 1), (100, 1), (500, 1), (500, 1))

In [23]:
gp = GaussianProcessRBF()
gp.fit(X_train, y_train)